In [1]:
import os
# 在继续之前，请确保已经设置了用于调用大模型的 API Key。
# 一般来说，只需设置 application.yaml 中 llm.qwen.api_key_env 指向的环境变量即可。
#
# 例如：
#   在 Windows:
#       set API_KEY_QWEN=你的密钥
#   在 macOS / Linux:
#       export API_KEY_QWEN=你的密钥
#
# 如未设置，将无法构建向量库（embedding 会依赖该 Key）。

env_name = "API_KEY_QWEN"
if not os.environ.get(env_name):
    print(f"注意：尚未检测到环境变量 {env_name}，如需构建向量库，请先设置它。")

# VectorDatabaseBuilder 会自动读取 config/application.yaml 与 model_roles.yaml，
# 完成数据加载 → 样本生成 → 文本切片 → 构建向量库 等全部步骤。
# 用户无需关注内部细节，只需调用 invoke() 即可。

from langchain.schema import Document

def pretty_print_docs(docs, max_chars=200):
    """
    将检索结果以整洁的方式展示给用户（极其适合 quickstart）。
    """
    print(f"检索到 {len(docs)} 条文档：\n")
    for idx, doc in enumerate(docs, 1):
        content = doc.page_content.replace("\n", " ").strip()
        snippet = content[:max_chars] + ("..." if len(content) > max_chars else "")
        print(f"[{idx}] {snippet}\n")



In [ ]:
from rag_eval import VectorDatabaseBuilder,RagRunner
question = "《战国无双3》是由哪两个公司合作开发的？"

vector_manager = VectorDatabaseBuilder().invoke() # 建立数据库、返回数据库管理器

In [5]:
docs = vector_manager.invoke(question)
pretty_print_docs(docs)

[vector_store_manager] invoke() 检索完成：collection = 'cmrc2018_default', k = 3, 命中文档数 = 3
检索到 3 条文档：

[1] 《战国无双3》（）是由光荣和ω-force开发的战国无双系列的正统第三续作。本作以三大故事为主轴，分别是以武田信玄等人为主的《关东三国志》，织田信长等人为主的《战国三杰》，石田三成等人为主的《关原的年轻武者》，丰富游戏内的剧情。此部份专门介绍角色，欲知武器情报、奥义字或擅长攻击类型等，请至战国无双系列1.由于乡里大辅先生因故去世，不得不寻找其他声优接手。从猛将传 and Z开始。2.战国无双 编年...

[2] 。但游戏内战役数量繁多，部分地图会有兼用的状况，战役虚实则是以光荣发行的2本「战国无双3 人物真书」内容为主，以下是相关介绍。（注：前方加☆者为猛将传新增关卡及地图。）合并本篇和猛将传的内容，村雨城模式剔除，战国史模式可直接游玩。主打两大模式「战史演武」&「争霸演武」。系列作品外传作品

[3] 。广茂铁路是在2004年2月29日起，因广州铁路集团控股的三茂铁路股份有限公司管辖范围增加原由广铁集团羊城铁路总公司管辖的广三铁路，原广三铁路和三茂铁路合称广茂铁路，原属广三线上的广州西站、石围塘站、佛山车务段和工务、电务、水电等部门由三茂铁路股份有限公司接管。同时中华人民共和国铁道部将广茂铁路由二级线路调整为一级线路。广三铁路于1903年筑成，全长49公里。而全长357公里的三茂铁路则于1991...



In [6]:
runner = RagRunner(vector_manager,rag_type = "normal_rag")
result = runner.invoke(question)
result


{'question': '《战国无双3》是由哪两个公司合作开发的？',
 'answer': '《战国无双3》是由**光荣**（Koei）和**ω-force**两个公司合作开发的。',
 'contexts': [{'content': '《战国无双3》（）是由光荣和ω-force开发的战国无双系列的正统第三续作。本作以三大故事为主轴，分别是以武田信玄等人为主的《关东三国志》，织田信长等人为主的《战国三杰》，石田三成等人为主的《关原的年轻武者》，丰富游戏内的剧情。此部份专门介绍角色，欲知武器情报、奥义字或擅长攻击类型等，请至战国无双系列1.由于乡里大辅先生因故去世，不得不寻找其他声优接手。从猛将传 and Z开始。2.战国无双 编年史的原创男女主角亦有专属声优。此模式是任天堂游戏谜之村雨城改编的新增模式。本作中共有20张战场地图（不含村雨城），后来发行的猛将传再新增3张战场地图',
   'metadata': {'doc_id': 'DEV_0_QUERY_0_0', 'sample_id': 'DEV_0_QUERY_0'}},
  {'content': '。但游戏内战役数量繁多，部分地图会有兼用的状况，战役虚实则是以光荣发行的2本「战国无双3 人物真书」内容为主，以下是相关介绍。（注：前方加☆者为猛将传新增关卡及地图。）合并本篇和猛将传的内容，村雨城模式剔除，战国史模式可直接游玩。主打两大模式「战史演武」&「争霸演武」。系列作品外传作品',
   'metadata': {'doc_id': 'DEV_0_QUERY_0_1', 'sample_id': 'DEV_0_QUERY_0'}},
  {'content': '。广茂铁路是在2004年2月29日起，因广州铁路集团控股的三茂铁路股份有限公司管辖范围增加原由广铁集团羊城铁路总公司管辖的广三铁路，原广三铁路和三茂铁路合称广茂铁路，原属广三线上的广州西站、石围塘站、佛山车务段和工务、电务、水电等部门由三茂铁路股份有限公司接管。同时中华人民共和国铁道部将广茂铁路由二级线路调整为一级线路。广三铁路于1903年筑成，全长49公里。而全长357公里的三茂铁路则于1991年投入运营。',
   'metadata': {'doc_id': 'DEV_2_QUERY_0_1', 'sample_id': 'DEV